In [1]:
import pandas as pd
datafile= '/Users/z003zhj/desktop/Recommendation system/BX-CSV-Dump/BX-Book-Ratings.csv'
data = pd.read_csv(datafile, sep=";", header=0, names=["user", "isbn", "rating"], encoding='cp1252')

In [10]:
data.head()


,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [2]:
bookfile= '/Users/z003zhj/desktop/Recommendation System/BX-CSV-Dump/BX-Books.csv'
books = pd.read_csv(bookfile, sep=";", header=0, error_bad_lines=False, usecols=[0,1,2], index_col=0, names=["isbn", "title", "author"], encoding='cp1252')


In [12]:
books.head()


,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [13]:
def bookmeta(isbn):
    title = books.at[isbn, "title"]
    author= books.at[isbn, "author"]
    return title, author

In [14]:
bookmeta("0393045218")

('The Mummies of Urumchi', 'E. J. W. Barber')

In [15]:
def favbooks(user, N):
    userratings = data[data["user"]==user]
    sortedratings = pd.DataFrame.sort_values(userratings, ['rating'], ascending=[0])[:N]
    sortedratings["title"] = sortedratings["isbn"].apply(bookmeta)
    return sortedratings

In [16]:
data = data[data["isbn"].isin(books.index)]

In [17]:
data.shape

(1031175, 3)

In [18]:
favbooks(204622, 5)

,user,isbn,rating,title
844955,204622,0967560500,10,"(Natural Hormonal Enhancement, Rob Faigin)"
844935,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
844926,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
844958,204622,097173660X,9,"(Life After School Explained, Cap &amp; Compass)"
844920,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"


In [19]:
distinctisbn = data.isbn.value_counts()
distinctisbn.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [20]:
distinctisbn.shape


(270170,)

In [21]:
distinctuser= data.user.value_counts()
distinctuser.head(10)

11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
212898     4290
278418     3996
76352      3329
110973     2971
235105     2943
Name: user, dtype: int64

In [22]:
distinctuser.shape


(92107,)

In [23]:
data = data[data["isbn"].isin(distinctisbn[distinctisbn>10].index)]

In [38]:
data = data[data["user"].isin(distinctuser[distinctuser>10].index)]

In [24]:
useritemratingmatrix= pd.pivot_table(data, values='rating', index=['user'], columns=['isbn'])

In [26]:
user1 = 204622
user2 = 255489

In [27]:
user1ratings = useritemratingmatrix.transpose()[user1]
user1ratings.head()

isbn
0002005018   NaN
0002251760   NaN
0002259834   NaN
0002558122   NaN
0006480764   NaN
Name: 204622, dtype: float64

In [29]:
user2ratings = useritemratingmatrix.transpose()[user2]

In [30]:
from scipy.spatial.distance import hamming
hamming(user1ratings, user2ratings)

0.999935283458452

In [33]:
import numpy as np
def distance(user1, user2):
    try:
        u1 = useritemratingmatrix.transpose()[user1]
        u2 = useritemratingmatrix.transpose()[user2]
        distance = hamming(u1, u2)
    except:
        distance = np.NaN
    return distance
    

In [34]:
distance(278418, 278554)

1.0

In [35]:
user = 204622
allusers = pd.DataFrame(useritemratingmatrix.index)
allusers = allusers[allusers["user"]!=user]

In [36]:
allusers.head()


,user
0,8
1,9
2,10
3,14
4,16


In [37]:
allusers["distance"] = allusers["user"].apply(lambda x: distance(user, x))

In [38]:
allusers.head()

,user,distance
0,8,1.000000
1,9,1.000000
2,10,1.000000
3,14,0.999935
4,16,1.000000


In [39]:
K=10
Knearestusers = allusers.sort_values(["distance"], ascending=True)["user"][:K]

In [40]:
Knearestusers


19018     82893
15886     68555
20052     87555
32176    140036
57732    251422
52994    232131
45250    198711
11085     48046
3578      16795
1605       7346
Name: user, dtype: int64

In [41]:
def nearestneighbors(user, K=10):
    allusers = pd.DataFrame(useritemratingmatrix.index)
    allusers = allusers[allusers["user"]!=user]
    allusers["distance"] = allusers["user"].apply(lambda x: distance(user, x))
    Knearestusers = allusers.sort_values(["distance"], ascending=True)["user"][:K]
    return Knearestusers

In [42]:
nearestneighbors(user)

19018     82893
15886     68555
20052     87555
32176    140036
57732    251422
52994    232131
45250    198711
11085     48046
3578      16795
1605       7346
Name: user, dtype: int64

In [59]:
Knearestusers = nearestneighbors(user)
NNratings = useritemratingmatrix[useritemratingmatrix.index.isin(Knearestusers)]
NNratings

isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
7346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
